In [1]:
from facebook_business.api import FacebookAdsApi
from facebook_business.exceptions import FacebookRequestError
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adreportrun import AdReportRun
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.adaccountuser import AdAccountUser as AdUser
from facebook_business import adobjects
from matplotlib import pyplot as plt
import pandas as pd
from datetime import date, timedelta
import time as t

In [2]:
import requests
import json

In [3]:
import httplib2
import argparse
from googleapiclient import discovery
from googleapiclient.discovery import build
from oauth2client import client
from oauth2client import tools
from oauth2client import file
from oauth2client.file import Storage
import google.oauth2.credentials
import google_auth_oauthlib.flow
from oauth2client.service_account import ServiceAccountCredentials
import gspread

#### `gspread` and Google spreadsheets

In [4]:
SCOPES = ['https://spreadsheets.google.com/feeds',  
          'https://www.googleapis.com/auth/drive']

CREDENTIALS = ServiceAccountCredentials.from_json_keyfile_name(
    r'.../google-sheets-service-key.json', SCOPES)

http = CREDENTIALS.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?version=v4')
service = discovery.build('sheets', 'v4', http = http, discoveryServiceUrl=discoveryUrl)

gc = gspread.authorize(CREDENTIALS)
sh = gc.open('CWA budgets reporting')

spreadsheet_id = '1_pgxcPDlFlCupgp2pmFdLBoXJQAymgpfIz0AVV7XshY'

* `Marketing API` > `Tools`
* `Tools` > `Graph API Explorer` `access token`
* `Access Token Debugger` > `Expand Access Token`

In [5]:
my_access_token = '...'
my_app_id = '...'
my_app_secret = '...'

In [6]:
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token, api_version='v14.0')
me = AdUser(fbid='me')
my_accounts = list(me.get_ad_accounts())

#### Token expiring time

In [7]:
time = requests.get(f'https://graph.facebook.com/oauth/access_token_info?client_id={my_app_id}&access_token={my_access_token}').json()
time = timedelta(seconds=time['expires_in'])
print(f'Access token expires in: {time}')

Access token expires in: 36 days, 22:44:32


#### Getting list of accounts

In [8]:
account_ids = {}
i = 0
while i < len(my_accounts):
    my_account = my_accounts[i]
    name = my_account.api_get(fields=[my_account.Field.name])['name']
    account_id = my_account.api_get(fields=[my_account.Field.name])['account_id']
    account_ids[name] = account_id
    i += 1

In [9]:
fields = [
    AdsInsights.Field.campaign_id,
    AdsInsights.Field.clicks,
    AdsInsights.Field.spend,
    AdsInsights.Field.impressions]

count = 0

def wait_for_async_job(async_job):
    global count
    async_job = async_job.api_get()
    while async_job[AdReportRun.Field.async_status] != 'Job Completed' or async_job[
        AdReportRun.Field.async_percent_completion] < 100:
        t.sleep(2)
        async_job = async_job.api_get()
    else:
        print('Job ' + str(count) + ' completed')
        count += 1
    return async_job.get_result(params={'limit': 1000})

def get_insights(account, date_preset='last_3d'):
    account = AdAccount(account['id'])
    i_async_job = account.get_insights(
        params={
            'level': 'ad',
            'date_preset': date_preset,
            'time_increment': 1},
            fields=fields,
            is_async=True)
    results = [dict(item) for item in wait_for_async_job(i_async_job)]
    return results

In [10]:
d = {}

i = 0
while i < len(my_accounts):
    account_id = my_accounts[i]['account_id']
    if account_id in account_ids.values():
        
        r = get_insights(my_accounts[i], 'this_month')
        
        impressions = 0
        clicks = 0
        spend = 0
        for x in r:
            impressions += int(x['impressions'])
            clicks += int(x['clicks'])
            spend += float(x['spend'])
            
        d[list(account_ids.keys())[list(account_ids.values()).index(account_id)]] = [str(date.today().replace(day=1)), 
                                                                                     str(date.today()), 
                                                                                     impressions, 
                                                                                     clicks, 
                                                                                     spend]
    
    i += 1
    
print('Facebook ads reporting: https://docs.google.com/spreadsheets/d/1_pgxcPDlFlCupgp2pmFdLBoXJQAymgpfIz0AVV7XshY/edit#gid=279541624')

Job 0 completed
Job 1 completed
Job 2 completed
Job 3 completed
Job 4 completed
Job 5 completed
Job 6 completed
Job 7 completed
Job 8 completed
Job 9 completed
Job 10 completed
Job 11 completed
Job 12 completed
Job 13 completed
Job 14 completed
Job 15 completed
Job 16 completed
Job 17 completed
Job 18 completed
Job 19 completed
Facebook ads reporting: https://docs.google.com/spreadsheets/d/1_pgxcPDlFlCupgp2pmFdLBoXJQAymgpfIz0AVV7XshY/edit#gid=279541624


In [11]:
data = pd.DataFrame(d).T
data.columns = ['date_start', 'date_stop', 'impressions', 'clicks', 'ad_cost']
data

,date_start,date_stop,impressions,clicks,ad_cost
Fjarn - 15124416,2022-07-01,2022-07-31,0,0,0
Dmitry Egoshin,2022-07-01,2022-07-31,0,0,0
Comrade Web Agency,2022-07-01,2022-07-31,110086,731,3298.39
SNAC System Business Account,2022-07-01,2022-07-31,0,0,0
Hot Ground Gym®,2022-07-01,2022-07-31,34716,1184,335.91
Cultivate Advisors,2022-07-01,2022-07-31,310644,1912,23657.72
Hot Ground Gym New,2022-07-01,2022-07-31,0,0,0
Quadwalls FB Ad Account,2022-07-01,2022-07-31,0,0,0
American Tent Ads,2022-07-01,2022-07-31,73859,3595,749.12
Earth Development,2022-07-01,2022-07-31,0,0,0


In [12]:
service.spreadsheets().values().clear(spreadsheetId=spreadsheet_id, range='Sheet3!A2:K').execute()

for key, value in d.items():
    body = {'values': [[key, value[0], value[1], value[2], value[3], value[4]]]}
    
    service.spreadsheets().values().append(spreadsheetId=spreadsheet_id, range='Sheet3!A2:K', 
                                       valueInputOption='USER_ENTERED', 
                                       body=body).execute()
print('Spreadsheet is ready')

Spreadsheet is ready
